# Determine Meaning of Date Fields
Find a field with a date string convert it to ANSI and determine if it is likely to be an account expiry date, a birth date, or a date which can be ignored.

While birth dates can range from today, for very keen moms to up to 100 years ago, most will be at least a few years ago. Actually if the date is in the past it can be a birth date, and only dates in the future can be expiry dates. If there is no future date in the data the account was not assigned an expiry.

**Note**:
Dates are only valid in the following formats: `yyyy-mm-dd, yyyy/mm/dd, mm-dd-yyyy, mm/dd/yyyy, yyyymmdd, mmddyyyy`.

In [28]:
import datetime
import re

def getDate(some_date:str):
    """
    Guesses if the argument is a date field and if it is, is it a birth date, expiry, or neither.
    For example:
    >>> getDate("23-12-2020")
    '20201223'
    >>> getDate("12-23-2020")
    '20201223'
    >>> getDate("12232020")
    '20201223'
    >>> getDate("2020-12-23")
    '20201223'
    >>> getDate("2020")
    
    >>> getDate("2020-12-23 19:37:10 GMT")
    '20201223'
    >>> getDate("2020-12-23T19:37:10.12234 GMT")
    '20201223'
    >>> getDate("2022-11-06 18:02:01.558085")
    '20221106'
    >>> getDate("Bar")
    
    >>> getDate("2022-31-06 18:02:01.558085")
    
    >>> getDate("31-06-2022 18:02:01.558085")
    
    """
    # Covers: yyyy-mm-dd, yyyy/mm/dd, mm-dd-yyyy, mm/dd/yyyy, yyyymmdd, mmddyyyy
    reg_mmddyyyy = re.compile(r'^(0[1-9]|1[012])[-\/]?(0[1-9]|[12][0-9]|3[01])[-\/]?(19|20)\d\d')
    reg_ddmmyyyy = re.compile(r'^(0[1-9]|[12][0-9]|3[01])[-\/]?(0[1-9]|1[012])[-\/]?(19|20)\d\d')
    reg_yyyymmdd = re.compile(r'^(19|20)\d\d[-\/]?(0[1-9]|1[012])[-\/]?(0[1-9]|[12][0-9]|3[01])')
    test_date = some_date
    new_date = ""
    for ch in ['\/','-']:
        if ch in some_date:
            some_date = some_date.replace(ch,'')
    # Trim off any trailing timestamps.
    some_date = some_date[:8]
#     print(f"some_date::{some_date}")
    if reg_mmddyyyy.match(test_date):
#         print(f"matches mmddyyyy format.")
        try:
            new_date = datetime.datetime.strptime(some_date, "%m%d%Y").date()
        except ValueError:
            return
    elif reg_ddmmyyyy.match(test_date):
#         print(f"matches ddmmyyyy format.")
        try:
            new_date = datetime.datetime.strptime(some_date, "%d%m%Y").date()
        except ValueError:
            return
    elif reg_yyyymmdd.match(test_date):
#         print(f"matches yyyymmdd format.")
        try:
            new_date = datetime.datetime.strptime(some_date, "%Y%m%d").date()
        except ValueError:
            return
    else:
#         print(f"{test_date} doesn't look like a date.")
        return
    yyyymmdd = ''.join(c for c in str(new_date) if c not in '-')[0:8]
    return yyyymmdd

# Tests if a string is likely to be an expiry date.
# return: True if the date is sometime in the future, and false otherwise.
def isExpiry(some_date:str):
    """
    Tests is supplied date is a reasonable expiry date. Reasonable is anywhere from tomorrow 
    on up.
    >>> isExpiry("2023-01-01")
    True
    >>> isExpiry("2043-01-01")
    True
    >>> isExpiry("2018-01-01")
    False
    >>> today = datetime.datetime.now()
    >>> isExpiry(f"{today}")
    False
    >>> tomorrow = datetime.datetime.now() + datetime.timedelta(20)
    >>> isExpiry(f"{tomorrow}")
    True
    >>> isExpiry("Foo")
    False
    """
    test_date = getDate(some_date)
    if test_date == None:
        return False
    tomorrow = datetime.datetime.now() + datetime.timedelta(1)
    possible_expiry = datetime.datetime.strptime(test_date, "%Y%m%d")
    if possible_expiry > tomorrow:
        return True
    return False
    

# Tests if a string is likely to be a birth date.
# return: True if the date is in the past on the order of years past, and false otherwise.
def isBirthDate(some_date:str):
    """
    Tests if supplied date is in the medium to far past. Far past is no longer than 114 years and no less than 
    1 year, though that can potentially be problematic.
    >>> isBirthDate("23-12-2019")
    True
    >>> isBirthDate("01-01-1888")
    False
    >>> isBirthDate("1963-08-22")
    True
    >>> isBirthDate("Toast")
    False
    """
    test_date = getDate(some_date)
    if test_date == None:
        return False
    two_years = 2 * 365
    one_hundred_years = 100 * 365
    two_years_ago = datetime.datetime.now() - datetime.timedelta(two_years)
    age = datetime.datetime.strptime(test_date, "%Y%m%d")
    if age < two_years_ago:
        one_hundred_years_ago = datetime.datetime.now() - datetime.timedelta(one_hundred_years)
        if age > one_hundred_years_ago:
            return True
    return False
    
if __name__ == "__main__":
    import doctest
    doctest.testmod()